# Системы машинного обучения в Production

- - -
## Содержание<a class="anchor" id="toc"></a>
* [Содержание](#toc)
* [Введение в проект](#intro)
    * [Постановка задачи](#intro_task)
    * [Описание сырых данных](#intro_description)
* [Составление тренировочного датасета](#compose)
    * [Обзор данных](#compose_overview)
    * [Объединение в единый датасет](#compose_join)
    * [Приведение типов](#compose_cast)
* [Препроцессинг](#preprocess)
    * [Обаботка категориальных признаков](#preprocess_categories)
    * [Обработка выбросов](#preprocess_outliers)
    * [Обработка пропусков](#preprocess_gaps)
* [Анализ данных](#analysis)
    * [Анализ целевой переменной](#analysis_target)
    * [Анализ признакового пространства](#analysis_corr)
* [Обучение на данных](#ml)
    * [Отбор признаков](#ml_feature_selection)
    * [Нормализация](#ml_normalize)
    * [Разбиение на train и test](#ml_split)
    * [Балансировка классов](#ml_balancing)
    * [Подбор моделей, получение бейзлана](#ml_model_selection)
    * [Выбор наилучшей модели, настройка гиперпараметров](#ml_hyper)
    * [Проверка качества, борьба с переобучением](#ml_overfittig)
    * [Сохранение модели](#ml_persist)
* [Экспорт результатов](#export)
    * [Загрузка private выборки](#export_load)
    * [Трансформация датасета](#export_transformation)
    * [Предсказание результатов](#export_prediction)
    * [Сохранение результатов](#export_predicted)
* [Интерпретация результатов](#interpretation)

- - -
## Введение в проект<a class="anchor" id="intro"></a>

### Постановка задачи<a class="anchor" id="task"></a>

**Задача**
1. Необходимо, используя сырые данные игровых логов, собрать датасет, предобработать его и обучить модель, предсказывающую уход игрока из проекта.
2. Аналогичным с обучающей выборкой образом нужно сформировать признаковое описание игроков для тестовой выборки и сделать прогноз для игроков из тестового датасета.
3. Проект оформить в виде *.py модулей и скриптов проекта в PyCharm, где функционал и все этапы решения задачи разнесены по модулям.

**Наименование файлов с данными**
* train/\*.csv — файлы для обучающего датасета
* test/\*.csv — файлы для тестового датасета

**Целевая переменная**
**is_churned** - уход игрока из проекта

**Метрика качества**
F1-score

**Требования к решению**
* F1 > 0.4
* Метрика оценивается по качеству прогноза для главного класса (1 - уход из игры)
* Jupyter Notebook с кодом решения
* CSV-файл с прогнозами (два столбца: user_id | is_churned)
* Ход решения должен быть последовательным, сопровождаться комментариями

**Сроки сдачи**
Cдать проект нужно в течение 5 дней после окончания последнего вебинара. Оценки работ, сданных до дедлайна, будут представлены в виде рейтинга, ранжированного по заданной метрике качества. Проекты, сданные после дедлайна или сданные повторно, не попадают в рейтинг, но можно будет узнать результат.

### Описание сырых данных<a class="anchor" id="description"></a>

**profiles** — профили
* user_id — идентификатор игрока (во всех остальных файлах имеет тот же смысл)
* age — возраст (получен из даты рождения, что указана в БД)
* gender — пол (M – Male, F – Female)
* days_between_reg_fl — дней между регистрацией и первым входом
* days_between_fl_df — дней между первым входом и первым платежом
* has_return_date — есть дата возврата (игрок когда-то уходил и возвращался)
* has_phone_number — есть привязка номера телефона к аккаунту

**payments** — платежи
* log_dt — дата события (во всех остальных файлах имеет тот же смысл)
* pay_amt — размер внесенных средств (сумма платежей за данную дату)
* trans_amt — кол-во транзакций за данную даты

**reports** — жалобы игрока на других пользователей
* reports_amt — кол-во жалоб (на читы/оскорбления)
* abusers — сессии с нарушителями
* sess_with_abusers_amt — кол-во сессий с теми, на кого жаловались другие игроки

**logins** — входы
* disconnect_amt — кол-во завершений игры из-за технической проблемы
* session_amt — кол-во сессий (от входа в клиент игры до выхода)

**pings** — пинг до серверов (чем меньше, тем лучше интернет соединение и игра меньше "глючит")
* avg_min_ping — средний за данную дату пинг до ближайшего сервера

**sessions** — информация о сессиях (матчах)
* kd — Kills/Deaths — отношение убийств к смертям, то есть насколько успешно играл пользователь
* win_rate — Wins/Matches — отношение кол-во матчей, закончившихся победой, к общему числу матчей, тоже мера успешности
* leavings_rate — Leavings/Matches — отношение кол-во матчей, покинутых игроком, к общему числу матчей
* session_player — суммарное время в матчах за данную дату (в секундах)

**shop** — активность в магазине
* silver_spent — траты серебра
* gold_spent — траты золота

- - -
## Обзор данных<a class="anchor" id="overview"></a>